In [ ]:
# This was my final cropping script. It was used to crop
# the Garden dataset. It is able to take in images of
# arbitrary size, as I was manually cropping a few images to remove
# the edge of the mirror, my feet, etc. It also requires 
# JSON annotations of squares around the target (laptop), although
# it only uses the left and right coordinates of the image.

# As the gardent dataset was taken much closer to the laptop,
# it was best to have all crops contain the full vertical height.
# This script aimed to make crops to maximize the horizontal diversity
# (preferring more crops) while still keeping the target in frame.

# The end product makes either 1, 2 or 3 crops, depending on the image.
# If the target takes up most of the image, only crop is made, or it
# the target is close to an edge, 1 or 2 crops are made. If the target
# is small enough compared to the image, 3 crops are made, with the 
# target on the left, in the middle, and on the right, respectively.

In [2]:
from PIL import Image 
import glob
import os
import random
import json

In [104]:
def extract_sides(annotation):
    # Returns a tuple of the side coordinates of the box from an annotation
    # from Sloth
    
    box_xl = int(annotation["x"])
    box_xr = int(box_xl + annotation["width"])
    
    return box_xl, box_xr

def make_crop_coordinates(box_coords, width, height):
    
    # Get image dimensions and annotation dimensions
    img_xl = 0
    img_xr = width
    
    box_xl = box_coords[0]
    box_xr = box_coords[1]
        
    square_dim = height
    
    # If target is close to the left edge
    if box_xl <= 45:
        # crop once, left side
        crop_one = (0, square_dim)
        
        return [crop_one]
        
    # If target is close to the right edge
    elif box_xr >= img_xr - 45:
        # crop once, right side
        crop_one = (img_xr - square_dim, img_xr)
        
        return [crop_one]
    
    # If the target is bigger than or close to the square dim
    # just do one center crop
    elif box_xr - box_xl >= square_dim-45:
        center = box_xl + ((box_xr - box_xl) / 2)
        crop_one = (int(center - square_dim/2),int((center - square_dim/2) + square_dim))
        
        return [crop_one]
         
    else:        
        crop_one = (box_xl -45, box_xl-45 + square_dim)
        # If the crop is over the edge, change it
        if crop_one[1] > img_xr:
            crop_one = (img_xr - square_dim, img_xr)
                 
        crop_two = (box_xr+45 - square_dim, box_xr+45)
        # If the crop is over the edge, change it
        if crop_two[0] < 0:
            crop_two = (0, square_dim)
        
        center = box_xl + ((box_xr - box_xl) / 2)
        crop_three = (int(center - square_dim/2),int((center - square_dim/2) + square_dim))
        # If the center crop is going of the image
        # we've already done an edge so drop it
        if crop_three[1] > img_xr or crop_three[0] < 0:
            crop_three = None
        
        # Assert all crops are correct
        for crop in [crop_one, crop_two, crop_three]:
            if crop:
                assert crop[0] >= 0 and crop[1] <= img_xr, "%s, %s, %s" % (str(crop_one),str(crop_two),str(crop_three))
                assert crop[0] <= box_xl and crop[1] >= box_xr
                
        return [crop for crop in [crop_one,crop_two,crop_three] if crop]
                    
    

In [118]:
# The manually cropped images where saved as RGBA, which can't save as jpeg
# this fixed it
def pure_pil_alpha_to_color_v2(image, color=(255, 255, 255)):
    """Alpha composite an RGBA Image with a specified color.

    Simpler, faster version than the solutions above.

    Source: http://stackoverflow.com/a/9459208/284318

    Keyword Arguments:
    image -- PIL RGBA Image object
    color -- Tuple r, g, b (default 255, 255, 255)

    """
    image.load()  # needed for split()
    background = Image.new('RGB', image.size, color)
    background.paste(image, mask=image.split()[3])  # 3 is the alpha channel
    return background

In [121]:
# Set the path settings here
# which have to match the locations, etc.
# from Sloth

path = "manual_cropped/"
save_path = "manual_cropped_ac/"
annotation_file = path + "manual_cropped.json"

def main(path, save_path, annotation_file, RGBA = False, j = 0):
    
    # Get the annotation dic
    annotation_dic = {}

    # Load the annotations into a dictionary,
    # accesible by the file name key
    with open(annotation_file) as json_file:
        annotations = json.load(json_file)
        for i in range(len(annotations)):
            annotation_dic[annotations[i]['filename']] = annotations[i]['annotations']     
        
    # Start the crop loop
    
    for fn in glob.glob( path + '*.png'):
        new_ims = []
        # Load an image
        with Image.open(fn) as im:
            # If its RGBA, fix it
            if RGBA:
                im = pure_pil_alpha_to_color_v2(im)

            # get the dimensions of the image
            width, height = im.size

            # Get the annotation
            annotation = annotation_dic[fn[len(path):]][0]

            # Get the x coords
            box_coords = extract_sides(annotation)

            # Propose crops
            crop_coords = make_crop_coordinates(box_coords, width, height)

            # Add the succesful crops
            for i in range(len(crop_coords)):
                full_crop_coords = (crop_coords[i][0],0,crop_coords[i][1],height)
                with im.crop(full_crop_coords) as new_im:
                    new_im = new_im.save(save_path+str(j) + "_cropped.jpg")
                    j+=1
    return j
                

j = main(path, save_path, annotation_file, RGBA = True)

path = "selfies/"
save_path = "selfies_ac/"
annotation_file = path + "selfies.json"

j = main(path, save_path, annotation_file, j = j)
